# Pill Classification using mobile net 

In [4]:
import os
import numpy as np
import re
import shutil

### Dataset preparation

In [10]:
current_directory = os.getcwd()
print("현재 경로:", current_directory)

현재 경로: /Users/seyeong/workspace/drugmaster_pillClassification/PillClassification


In [35]:
data_dir = 'test_augmented_images'
train = 'train'
validation = 'validation'
aug_data_dir = os.path.join(data_dir)
print(aug_data_dir)
image_dir = os.path.join(train, 'rectangle_img')
print(image_dir)
bbox_dir = os.path.join(train, 'rectangle_annotation')
print(bbox_dir)

test_augmented_images
train/rectangle_img
train/rectangle_annotation


In [16]:
for file in os.listdir(aug_data_dir):
#     print(file)
#     print(re.split('[/,-,(,). :]',file)[-1])
    file_format = re.split('[.]',file)[-1]
#     print(os.path.join(image_dir, file))
#     print(os.path.join(aug_data_dir, file))
    if file_format=='jpg':
        shutil.copy(os.path.join(aug_data_dir, file), os.path.join(image_dir, file))
    else:
        shutil.copy(os.path.join(aug_data_dir, file), os.path.join(bbox_dir, file))

In [17]:
## image file 수 확인
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']
print(len(image_files))
image_files[:5]

40000


['ApurtranTablet150mg(Irbesartan)_0_augmented_image_333.jpg',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_1_augmented_image_519.jpg',
 'GasridTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_826.jpg',
 'GasbetTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_480.jpg',
 'NumentaminSustainedReleaseCapsule8mg(GalantamineBromide)_0_augmented_image_678.jpg']

In [31]:
# class수 확인
class_list = set()
for image_file in image_files:
    file_name = re.split('_',image_file)[0]
    class_list.add(file_name)

class_list = list(class_list)
print(len(class_list))

20


In [32]:
class_list.sort()
class_list[:5]

['ApurtranTablet150mg(Irbesartan)',
 'BalsanTablet80mg(Valsartan)',
 'BalthrepTablet160mg(Valsartan)',
 'DiosartanTablet160mg(Valsartan)',
 'EscitalTablet5mg(EscitalopramOxalate)']

In [34]:
class2idx = {cls:idx for idx, cls in enumerate(class_list)}
class2idx['ApurtranTablet150mg(Irbesartan)']

0

In [39]:
train_images = os.listdir(image_dir)
val_images = os.listdir(validation)

print(len(train_images), len(val_images))

40000 16000


In [40]:
train_images[:10]

['ApurtranTablet150mg(Irbesartan)_0_augmented_image_333.jpg',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_1_augmented_image_519.jpg',
 'GasridTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_826.jpg',
 'GasbetTablet5mg(MosaprideCitrateHydrate)_1_augmented_image_480.jpg',
 'NumentaminSustainedReleaseCapsule8mg(GalantamineBromide)_0_augmented_image_678.jpg',
 'SurosinDTablet(TamsulosinHydrochloride)_1_augmented_image_433.jpg',
 'BalsanTablet80mg(Valsartan)_0_augmented_image_473.jpg',
 'ApurtranTablet150mg(Irbesartan)_0_augmented_image_73.jpg',
 'EscitalTablet5mg(EscitalopramOxalate)_1_augmented_image_355.jpg',
 'ValsartelTablet80mg(Valsartan)_1_augmented_image_631.jpg']

In [41]:
val_images[:10]

['BalthrepTablet160mg(Valsartan)_0_augmented_image_87.txt',
 'ApurtranTablet150mg(Irbesartan)_0_augmented_image_73.jpg',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_0_augmented_image_7.jpg',
 'RosorodTablet10mg(RosuvastatinCalcium)_1_augmented_image_37.jpg',
 'ZolpidemSustainedReleaseTablet(ZolpidemTartrate)_1_augmented_image_157.jpg',
 'GasdialTablet50mg(DimethiconeMagnesium)_1_augmented_image_177.jpg',
 'GasprenTablet(MosaprideCitrateDihydrate)_1_augmented_image_73.txt',
 'LipinonTablet80mg(AtorvastatinCalciumTrihydrate)_1_augmented_image_143.txt',
 'FeratracTablet2.5mg(Letrozole)_1_augmented_image_151.jpg',
 'SarvaltanTablet160mg(Valsartan)_0_augmented_image_130.jpg']

### TFRecord

In [42]:
IMG_SIZE = 100

In [43]:

## TFRecord 저장할 directory와 file 경로 설정
tfr_dir = os.path.join('tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'cls_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'cls_val.tfr')